In [1]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime

In [2]:
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [3]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [4]:
ov_me_sql = '''
    SELECT medical_education_number,type, medical_school, medical_school_state FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [5]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [6]:
MORE_SCHOOL_QUERY = \
    """
    SELECT
    PARTY_ID AS PARTY_ID_SCHOOL,
    KEY_VAL AS SCHOOL_ID
    FROM
    AMAEDW.PARTY_KEY
    WHERE
    KEY_TYPE_ID = 23
    """
more_school_ids = pd.read_sql(con=AMAEDW, sql=MORE_SCHOOL_QUERY)

In [7]:
SCHOOL_QUERY = \
    """
    SELECT
    P.PARTY_ID AS PARTY_ID_SCHOOL,
    P.KEY_VAL AS SCHOOL_ID,
    C.STATE_ID,
    C.COUNTRY_ID
    FROM
    AMAEDW.PARTY_KEY P, AMAEDW.POST_CD C, AMAEDW.PARTY_ADDR A
    WHERE
    P.KEY_TYPE_ID = 23
    AND
    P.PARTY_ID = A.PARTY_ID
    AND 
    A.POST_CD_ID = C.POST_CD_ID
    """
school_ids = pd.read_sql(con=AMAEDW, sql=SCHOOL_QUERY)

In [8]:
MED_SCHOOL_QUERY = \
    """
    SELECT
    SCH_PARTY_ID,
    STU_PARTY_ID AS PARTY_ID,
    GRAD_DT,
    STS_TYPE_ID,
    DEGREE_CD,
    GRAD_CONF_IND
    FROM
    AMAEDW.SCHOOL_ATT
    WHERE
    THRU_DT is null
    """
med_school = pd.read_sql(con=AMAEDW, sql=MED_SCHOOL_QUERY)

In [9]:
GME_QUERY = \
    """
    SELECT
    G.PARTY_HOSPITAL_ID,
    G.PERSON_PARTY_ID AS PARTY_ID,
    G.BEGIN_DT,
    G.END_DT,
    G.PRIM_SPEC_ID,
    G.SEC_SPEC_ID,
    G.TRAIN_TYPE,
    G.CONF_STS_ID,
    G.GME_STS_TYPE_CD,
    C.STATE_ID,
    C.COUNTRY_ID
    FROM
    AMAEDW.EDW_GME G, AMAEDW.POST_CD C, AMAEDW.PARTY_ADDR A
    WHERE
    G.THRU_DT is null
    AND
    G.PARTY_HOSPITAL_ID = A.PARTY_ID
    AND 
    A.POST_CD_ID = C.POST_CD_ID
    """
gme = pd.read_sql(con=AMAEDW, sql=GME_QUERY)

In [10]:
YEAR_QUERY = \
    """
    SELECT
    PARTY_ID, 
    PROG_YEAR,
    PROG_GRAD_YR
    FROM
    AMAEDW.MED_PROF
    WHERE
    THRU_DT is null
    """
gme_year = pd.read_sql(con=AMAEDW, sql=YEAR_QUERY)

In [11]:
ORG_QUERY = \
    """
    SELECT
    PARTY_ID AS PARTY_HOSPITAL_ID,
    ORG_NM, 
    THRU_DT
    FROM
    AMAEDW.ORG_NM
    """
org_names = pd.read_sql(con=AMAEDW, sql=ORG_QUERY)

In [12]:
org_names.THRU_DT = pd.to_datetime(org_names.THRU_DT)

In [13]:
org_names = org_names.sort_values('THRU_DT').drop_duplicates('PARTY_HOSPITAL_ID', keep='last')

In [14]:
school_ids = school_ids.drop_duplicates()

In [15]:
gme_singular = gme.drop_duplicates('PARTY_ID')

In [16]:
extra = more_school_ids[~more_school_ids.SCHOOL_ID.isin(school_ids.SCHOOL_ID)]
school_ids = pd.concat([school_ids, extra])

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [17]:
test = pd.merge(party_ids, ov_me[['medical_education_number', 'type']], left_on='ME', right_on='medical_education_number')
school_info = pd.merge(school_ids, org_names, left_on='PARTY_ID_SCHOOL', right_on='PARTY_HOSPITAL_ID', how='left').drop_duplicates()
med_with_info = pd.merge(med_school, school_info, left_on='SCH_PARTY_ID', right_on='PARTY_ID_SCHOOL', how='left').drop_duplicates()
gme_with_info = pd.merge(gme_singular, org_names, on='PARTY_HOSPITAL_ID', how='left').drop_duplicates()

In [18]:
date_list = []
for row in med_with_info.itertuples():
    try:
        new_date = datetime.datetime.strptime(str(row.GRAD_DT), '%Y-%m-%d')
    except:
        new_date = None
    date_list.append(new_date)
med_with_info['GRAD_DATE'] = date_list
med_with_info = med_with_info.sort_values('GRAD_DATE')

In [19]:
countries = []
statuses = []
for row in med_with_info.itertuples():
    country = 0
    status = 0
    if row.COUNTRY_ID == 6705:
        country = 1
    if row.STS_TYPE_ID == 9:
        status = 2
    if row.STS_TYPE_ID == 54:
        status = 1
    countries.append(country)
    statuses.append(status)
med_with_info['COUNTRY'] = countries
med_with_info['STATUS'] = statuses
med_with_info = med_with_info.sort_values(['STATUS','COUNTRY','GRAD_DATE'])

In [20]:
cc = med_with_info.drop_duplicates('PARTY_ID', keep=False)
DUPES = med_with_info[med_with_info.duplicated('PARTY_ID', keep=False)]
plz = DUPES.drop_duplicates('PARTY_ID', keep='last')
med_with_info_2 = pd.concat([plz, cc])

In [21]:
ALL = pd.merge(test, gme_year, on='PARTY_ID', how='left')
ALL = pd.merge(ALL, med_with_info_2, on='PARTY_ID',how='left')
ALL = pd.merge(ALL, gme_with_info, on='PARTY_ID', how='left', suffixes = ['_SCH', '_GME'])

In [22]:
today = datetime.date.today()
types = ['Physician','Student','Resident']
all_mes = list(ALL.ME)
us_schools = list(ALL[(ALL.COUNTRY_ID_SCH==6705)].ME)
us_gmes = list(ALL[(ALL.COUNTRY_ID_GME==6705)].ME)
elements = [
    {'data_element': 'Medical School Name',
     'element':'ORG_NM_SCH',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'Medical School State',
     'element':'STATE_ID_SCH',
    'null': ['',-1],
    'universe': us_schools},
    {'data_element': 'Medical School Country',
     'element':'COUNTRY_ID_SCH',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'Medical School Grad Date',
     'element':'GRAD_DT',
    'null': ['',-1, datetime.date(1900, 1, 30)],
    'universe': all_mes},
    {'data_element': 'Education Status',
     'element':'STS_TYPE_ID',
    'null': ['',-1,61,43],
    'universe': all_mes},
    {'data_element': 'Medical Degree',
     'element':'DEGREE_CD',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'GME Institution',
     'element':'ORG_NM_GME',
    'null': [''],
    'universe': all_mes},
    {'data_element': 'GME Institution State',
     'element':'STATE_ID_GME',
    'null': ['',-1],
    'universe': us_gmes},
    {'data_element': 'GME Begin Date',
     'element':'BEGIN_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME End Date',
     'element':'END_DT',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Primary Specialty',
     'element':'PRIM_SPEC_ID',
    'null': ['',-1,1883],
    'universe':all_mes},
    {'data_element': 'GME Secondary Specialty',
     'element':'SEC_SPEC_ID',
    'null': ['',-1,1883],
    'universe': all_mes},
    {'data_element': 'GME Training Type',
     'element':'TRAIN_TYPE',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Confirmation Status',
     'element':'CONF_STS_ID',
    'null': ['',-1],
    'universe': all_mes},
    {'data_element': 'GME Status Type',
     'element':'GME_STS_TYPE_CD',
    'null': ['',-1],
    'universe':all_mes},
    {'data_element': 'GME Year in Program',
     'element':'PROG_YEAR',
    'null': ['',-1],
    'universe':all_mes},
    {'data_element': 'GME Post Grad Year',
     'element':'PROG_GRAD_YR',
    'null': ['',-1],
    'universe':all_mes}
]
dict_list = []
for typo in types:
    total = len(ALL[ALL.type==typo])
    for element in elements:
        universe = len(ALL[(ALL.type==typo)&(ALL.ME.isin(element['universe']))])
        complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        true_complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isin(element['null']))&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [24]:
pd.DataFrame(dict_list).to_csv(f'../../Data/Measurement/Education_Completeness_{str(datetime.date.today())}.csv', index=False)

In [ ]:
datetime.datetime.strptime(today, '%Y-%m-%d')

In [25]:
import math
def coin_reward(minutes):
    x = 4+ 0.2*minutes + 5*(minutes-30)/30
    return math.floor(x)